In [1]:
import os
COMBINE_BYTES = 130*1024*1024
os.environ["XLA_FLAGS"] = f"--xla_gpu_simplify_all_fp_conversions --xla_gpu_all_reduce_combine_threshold_bytes={COMBINE_BYTES}"

import haiku as hk
import jax
import jax.numpy as jnp
import jmp
import model

In [2]:
# TODO: config dataclass
NUM_LAYERS = 24

def _forward(x):
    a = model.RWKV(num_layers=NUM_LAYERS, vocab_size=50277, n_embd=1024, dim_att=1024, dim_ffn=1024*4)
    return a(x)

# policy = jmp.get_policy('p=bf16,c=bf16,o=bf16')
# policy_wkv = jmp.get_policy('p=f32,c=f32,o=bf16')
# hk.mixed_precision.set_policy(_forward, policy)
# hk.mixed_precision.set_policy(model.RWKV, policy)
# hk.mixed_precision.set_policy(model.WKV, policy_wkv)

dummy_x = jnp.ones((1024), dtype=int)
rng_key = jax.random.PRNGKey(42)

In [3]:
from safetensors.numpy import load_file
import torch

#sd = load_file("../../RWKV-LM-deepspeed/RWKV-v4neo/RWKV-4-Pile-430M-20220808-8066.safetensors")
sd = torch.load("/home/mrsteyk/projects/RWKV-LM/RWKV-v4neo/RWKV-4-Pile-430M-20220808-8066.pth", map_location='cpu')
sd = {k: v.detach().cpu().float().numpy() for k,v in sd.items()}

# I am mentally ill, you cannot :clown: upon me
head = {"w": jax.device_put(sd["head.weight"].T)}
ln_out = {"scale": jax.device_put(sd["ln_out.weight"]), "offset": jax.device_put(sd["ln_out.bias"])}
emb = {"embeddings": jax.device_put(sd["emb.weight"])} # opt: put on jax.device("cpu")[0]?
ln0 = {"scale": jax.device_put(sd["blocks.0.ln0.weight"]), "offset": jax.device_put(sd["blocks.0.ln0.bias"])}

# Remember, no ~~Russians~~:clown: (how long until this old joke gets old again, ngl kinda miss those days even though scene's dead)
hsd = {"rwkv/head": head, "rwkv/ln0": ln0, "rwkv/emb": emb, "rwkv/ln_out": ln_out}
for i in range(NUM_LAYERS):
    hsd[f"rwkv/block_{i}/ln1"] = {"scale": jax.device_put(sd[f"blocks.{i}.ln1.weight"]), "offset": jax.device_put(sd[f"blocks.{i}.ln1.bias"])}
    hsd[f"rwkv/block_{i}/att"] = {
        "time_first": jax.device_put(sd[f"blocks.{i}.att.time_first"]),
        "time_decay": jax.device_put(sd[f"blocks.{i}.att.time_decay"]),
        "time_mix_k": jax.device_put(sd[f"blocks.{i}.att.time_mix_k"][0]),
        "time_mix_v": jax.device_put(sd[f"blocks.{i}.att.time_mix_v"][0]),
        "time_mix_r": jax.device_put(sd[f"blocks.{i}.att.time_mix_r"][0]),
    }
    hsd[f"rwkv/block_{i}/att/key"] = {"w": jax.device_put(sd[f"blocks.{i}.att.key.weight"].T)}
    hsd[f"rwkv/block_{i}/att/value"] = {"w": jax.device_put(sd[f"blocks.{i}.att.value.weight"].T)}
    hsd[f"rwkv/block_{i}/att/receptance"] = {"w": jax.device_put(sd[f"blocks.{i}.att.receptance.weight"].T)}
    hsd[f"rwkv/block_{i}/att/output"] = {"w": jax.device_put(sd[f"blocks.{i}.att.output.weight"].T)}

    hsd[f"rwkv/block_{i}/ln2"] = {"scale": jax.device_put(sd[f"blocks.{i}.ln2.weight"]), "offset": jax.device_put(sd[f"blocks.{i}.ln2.bias"])}
    hsd[f"rwkv/block_{i}/ffn"] = {
        "time_mix_k": jax.device_put(sd[f"blocks.{i}.ffn.time_mix_k"][0]),
        "time_mix_r": jax.device_put(sd[f"blocks.{i}.ffn.time_mix_r"][0]),
    }
    hsd[f"rwkv/block_{i}/ffn/key"] = {"w": jax.device_put(sd[f"blocks.{i}.ffn.key.weight"].T)}
    hsd[f"rwkv/block_{i}/ffn/value"] = {"w": jax.device_put(sd[f"blocks.{i}.ffn.value.weight"].T)}
    hsd[f"rwkv/block_{i}/ffn/receptance"] = {"w": jax.device_put(sd[f"blocks.{i}.ffn.receptance.weight"].T)}

# hsd = {k: {kk: vv.astype(jnp.float32) for kk,vv in v.items()} for k,v in hsd.items()}

# print(sd["blocks.0.att.time_decay"], sd["blocks.0.att.time_first"])

del head, ln_out, emb, ln0
del sd

In [4]:
from jax.config import config
config.update("jax_debug_nans", True)
config.update("jax_debug_infs", True)
config.update("jax_disable_jit", True)

a = hk.transform(_forward)

# Array([ -4.301072 , -21.066956 ,  -3.2056859, ...,  -2.9424925,
#         -3.7281127,  -3.1309748], dtype=float32)
a.apply(hsd, rng_key, x=[0]*8)[-1]

Array([ -4.387704 , -21.17562  ,  -3.3059247, ...,  -3.0609918,
        -3.8596807,  -3.1924791], dtype=float32)

In [ ]:
from jax.config import config
config.update("jax_debug_nans", True)
config.update("jax_debug_infs", True)
config.update("jax_disable_jit", True)

a = hk.transform(_forward)

i = a.init(rng=rng_key, x=dummy_x)
print("---")
i, a.apply(i, rng_key, x=dummy_x)
# i

In [2]:
jnp.arange(5), jnp.array([0,1,2,3,4])

(Array([0, 1, 2, 3, 4], dtype=int32), Array([0, 1, 2, 3, 4], dtype=int32))

In [3]:
# why does it need `,`
jnp.arange(5)[jnp.newaxis,:]

Array([[0, 1, 2, 3, 4]], dtype=int32)

In [4]:
jnp.array([[1, 2, 3], [4, 5, 6]]).T

Array([[1, 4],
       [2, 5],
       [3, 6]], dtype=int32)

In [9]:
# Time decay/first are monotonous arrays
a = jnp.array([1,2,3])
a = a[:,jnp.newaxis]
aa = jnp.expand_dims(a, 1)
aa, a, a.shape

(Array([[1],
        [2],
        [3]], dtype=int32),
 Array([[1],
        [2],
        [3]], dtype=int32),
 (3, 1))

In [10]:
T = 3 + 1
b = jnp.arange(-(T-2), 1)[jnp.newaxis, :]
# this should be ok for the operation?
matrix = jnp.concatenate([a*b, a], axis=1)
a, b, a*b, matrix 

(Array([[1],
        [2],
        [3]], dtype=int32),
 Array([[-2, -1,  0]], dtype=int32),
 Array([[-2, -1,  0],
        [-4, -2,  0],
        [-6, -3,  0]], dtype=int32),
 Array([[-2, -1,  0,  1],
        [-4, -2,  0,  2],
        [-6, -3,  0,  3]], dtype=int32))

In [7]:
@jax.jit
def wkv(w, u, k, v):
    T, C = k.shape
    time_curve = jnp.arange(-T+2, 1)[jnp.newaxis, ...]
    k, v = map(jnp.array, [[k], [v]])
    w = -jnp.exp(w)
    ek = jnp.exp(k.transpose((0, 2, 1)))
    ekv = ek * v.transpose((0, 2, 1))
    ew_time = jnp.expand_dims(jnp.exp(w), 1) * time_curve
    time_w = jnp.concatenate([ew_time, jnp.expand_dims(u, 1)], axis=1)
    w = jnp.expand_dims(jnp.exp(time_w), 1)

    # print(time_w.shape, ew_time.shape, time_w, ew_time)
    # print(ew_time.shape, time_w.shape, w.shape, ekv.shape, ek.shape)

    def pad(x): return jnp.pad(x, [(0, 0), (0, 0), (T-1, 0)])

    wkv = jax.lax.conv_general_dilated(pad(ekv), w, (1,), [(
        0, 0)], dimension_numbers=('NCW', 'OIW', 'NCW'), feature_group_count=C)
    wk = jax.lax.conv_general_dilated(pad(ek), w, (1,), [(
        0, 0)], dimension_numbers=('NCW', 'OIW', 'NCW'), feature_group_count=C)
    return (wkv / wk).transpose(0, 2, 1)[0].T

%timeit wkv(jnp.array([1. , 2., 3.]), jnp.array([0.1, 0.2, 0.3]), jnp.array([[11, 22, 33], [44, 55, 66], [77, 88, 99]], dtype=jnp.float32), jnp.array([[11, 22, 33], [44, 55, 66], [77, 88, 99]], dtype=jnp.float32))

564 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
%timeit model.WKV(jnp.array([1. , 2., 3.]), jnp.array([0.1, 0.2, 0.3]), jnp.array([[11, 22, 33], [44, 55, 66], [77, 88, 99]], dtype=jnp.float32), jnp.array([[11, 22, 33], [44, 55, 66], [77, 88, 99]], dtype=jnp.float32))

556 µs ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
